In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
pip install pyod

     |████████████████████████████████| 120 kB 4.2 MB/s 
     |████████████████████████████████| 38.1 MB 59.6 MB/s 
  Created wheel for pyod: filename=pyod-1.0.1-py3-none-any.whl size=147473 sha256=950719b3b81b7eb51d42a694e3e0b29326502118c24a078b6b9741f6605db930
  Stored in directory: /root/.cache/pip/wheels/ea/c4/29/67ad87835b209f72e4706369c683741b09490f2829d64ea768
Successfully built pyod
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pyod.models.knn import KNN
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [10]:
from google.colab import files
uploaded = files.upload()

Saving NBA_Dataset_A4.csv to NBA_Dataset_A4 (1).csv


In [11]:
dataset = pd.read_csv('NBA_Dataset_A4.csv')

In [12]:
dataset.head()
dataset.shape
dataset.dtypes
dataset.info()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.29,NYK,Knicks,68,"1,300.00","1,306.72",H,L,0.64,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.71,TRH,Huskies,66,"1,300.00","1,293.28",A,W,0.36,NaN
2,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.69,CHS,Stags,63,"1,300.00","1,309.65",A,L,0.37,NaN
3,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.01,NYK,Knicks,47,"1,306.72","1,297.07",H,W,0.63,NaN
4,3,194611020DTF,NBA,1,1947,11/2/1946,1,0,WSC,Capitols,...,43.14,DTF,Falcons,33,"1,300.00","1,279.62",A,W,0.36,NaN


(126314, 23)

gameorder          int64
game_id           object
lg_id             object
_iscopy            int64
year_id            int64
date_game         object
seasongame         int64
is_playoffs        int64
team_id           object
fran_id           object
pts                int64
elo_i            float64
elo_n            float64
win_equiv        float64
opp_id            object
opp_fran          object
opp_pts            int64
opp_elo_i        float64
opp_elo_n        float64
game_location     object
game_result       object
forecast         float64
notes             object
dtype: object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126314 entries, 0 to 126313
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   gameorder      126314 non-null  int64  
 1   game_id        126314 non-null  object 
 2   lg_id          126314 non-null  object 
 3   _iscopy        126314 non-null  int64  
 4   year_id        126314 non-null  int64  
 5   date_game      126314 non-null  object 
 6   seasongame     126314 non-null  int64  
 7   is_playoffs    126314 non-null  int64  
 8   team_id        126314 non-null  object 
 9   fran_id        126314 non-null  object 
 10  pts            126314 non-null  int64  
 11  elo_i          126314 non-null  float64
 12  elo_n          126314 non-null  float64
 13  win_equiv      126312 non-null  float64
 14  opp_id         126314 non-null  object 
 15  opp_fran       126314 non-null  object 
 16  opp_pts        126314 non-null  int64  
 17  opp_elo_i      126314 non-nul

#### 1. Check of logical incosistencies with respect to the 'pts', 'opp_pts', and 'game_result' columns. In basketball, a team wins if it scores more points than their opponent, and there are no ties allowed.

In [13]:
dataset.game_result.value_counts()

L    63157
W    63157
Name: game_result, dtype: int64

In [14]:
dataset.loc[((dataset['opp_pts'] < dataset['pts']) & (dataset['game_result'] == 'L')) |
            ((dataset['opp_pts'] > dataset['pts']) & (dataset['game_result'] == 'W')), ['opp_pts','pts','game_result']]

,opp_pts,pts,game_result
22,68,69,L
23,69,68,W
51979,114,97,W
52021,116,97,W
52094,114,117,L
87000,91,86,W
87001,86,91,L
87218,123,78,W
87219,78,123,L
87554,103,88,W


#### 2. Show all teams with large changes in their 'elo_i' ratings from 2014 to 2015

In [15]:
dataset[['team_id','elo_i','year_id']]

,team_id,elo_i,year_id
0,TRH,"1,300.00",1947
1,NYK,"1,300.00",1947
2,NYK,"1,306.72",1947
3,CHS,"1,300.00",1947
4,WSC,"1,300.00",1947
...,...,...,...
126309,GSW,"1,790.96",2015
126310,CLE,"1,704.39",2015
126311,GSW,"1,809.98",2015
126312,CLE,"1,700.74",2015


In [16]:
team_max_elo_2015 = dataset.loc[(dataset['year_id'] == 2015), ['elo_i']].groupby(dataset['team_id']).max().sort_values('team_id')
team_max_elo_2014 = dataset.loc[(dataset['year_id'] == 2014), ['elo_i']].groupby(dataset['team_id']).max().sort_values('team_id')

In [17]:
team_max_elo_2015.dropna(axis=0,inplace=True)
team_max_elo_2014.dropna(axis=0,inplace=True)

In [18]:
team_max_elo_2015['different_elo_i'] = team_max_elo_2015.elo_i - team_max_elo_2014.elo_i

In [19]:
# there the toppest different elo_i between two years 2014 and 2015 of each team
team_max_elo_2015.sort_values('different_elo_i', ascending = False).different_elo_i.head()

team_id
CLE   254.98
ATL   169.99
GSW   164.14
MIL    94.57
SAC    65.50
Name: different_elo_i, dtype: float64

In [20]:
team_max_elo_2015_e = dataset.loc[(dataset['year_id'] == 2015), ['elo_i']].groupby(dataset['team_id']).max().sort_values('team_id')
team_max_elo_2014_e = dataset.loc[(dataset['year_id'] == 2014), ['elo_i']].groupby(dataset['team_id']).max().sort_values('team_id')

In [21]:
team_max_elo_i = pd.concat([team_max_elo_2014_e,team_max_elo_2015_e])
team_max_elo_i['elo_i'].groupby(team_max_elo_i.index).max()

team_id
ATL   1,701.18
BOS   1,551.86
BRK   1,580.35
CHA   1,549.17
CHI   1,636.09
CHO   1,512.68
CLE   1,723.41
DAL   1,653.70
DEN   1,634.86
DET   1,477.78
GSW   1,813.63
HOU   1,676.47
IND   1,697.23
LAC   1,743.11
LAL   1,522.19
MEM   1,678.42
MIA   1,716.73
MIL   1,550.35
MIN   1,588.32
NOP   1,539.77
NYK   1,582.13
OKC   1,749.16
ORL   1,396.20
PHI   1,484.97
PHO   1,592.71
POR   1,667.73
SAC   1,512.10
SAS   1,759.68
TOR   1,634.73
UTA   1,565.02
WAS   1,588.25
Name: elo_i, dtype: float64

#### 3.Find any potential outlier teams that have won a large number away games.

In [22]:
dataset.game_location.value_counts()
dataset.game_result.value_counts()

H    63138
A    63138
N       38
Name: game_location, dtype: int64

L    63157
W    63157
Name: game_result, dtype: int64

In [23]:
dataset_team_W_A = dataset.loc[(dataset.game_location == 'A') & (dataset.game_result == 'W'), ['game_result']].groupby(dataset.team_id).count()
dataset_team_W_A.dropna(inplace=True)
dataset_team_W_A

,game_result
team_id,
ANA,8
AND,13
ATL,724
BAL,140
BLB,42
...,...
WAS,229
WAT,3
WSA,23


In [24]:
def IQR(data):
    first_q, third_q = data.quantile(0.25), data.quantile(0.75)
    OutlierRange = (third_q - first_q)*1.5
    high_outlier = third_q + OutlierRange
    low_outlier = first_q - OutlierRange
    return low_outlier, high_outlier

In [25]:
low_outlier, high_outlier = IQR(dataset_team_W_A)

In [26]:
dataset_team_W_A[(dataset_team_W_A > high_outlier) | (dataset_team_W_A < low_outlier)].dropna()

,game_result
team_id,
ATL,724.00
BOS,"1,356.00"
CHI,870.00
DET,924.00
HOU,787.00
LAL,"1,272.00"
MIL,800.00
NYK,"1,132.00"
PHI,957.00


In [27]:
dataset_team_W_A[dataset_team_W_A > (dataset_team_W_A.mean() + 3*dataset_team_W_A.std())].dropna()

,game_result
team_id,
BOS,"1,356.00"
LAL,"1,272.00"


In [28]:
def find_ZScore_Outliers(df, columnName):
    out=[]
    outValues=[]
    dfOrig = df
    df = df[columnName]
    m = np.mean(df)
    s = np.std(df)
    
    for val in df:
        z = (val - m) / s
        if (np.abs(z) > 3):
            out.append(val)
            outValues.append((dfOrig.loc[dfOrig[columnName] == val]).index.values[0])
            
    print("Outliers:", out)
    print("Rows of outliers:", outValues)
    tmpRes = zip(outValues, out)

    print (list(tmpRes))

In [29]:
find_ZScore_Outliers(dataset_team_W_A, 'game_result')

Outliers: [1356, 1272]
Rows of outliers: ['BOS', 'LAL']
[('BOS', 1356), ('LAL', 1272)]


#### 4. Fit a linear regression model for 'pts' (points scored) with respect to the 'elo_i' and 'elo_n' ratings. Identify teams with an outsized influence on the model. Plot your data

In [30]:
linear = ['pts','elo_i','elo_n']

In [31]:
dataset_linear = dataset.loc[:,linear].dropna()
dataset_linear.describe()

,pts,elo_i,elo_n
count,"126,314.00","126,314.00","126,314.00"
mean,102.73,"1,495.24","1,495.24"
std,14.81,112.14,112.46
min,0.00,"1,091.64","1,085.77"
25%,93.00,"1,417.24","1,416.99"
50%,103.00,"1,500.95","1,500.95"
75%,112.00,"1,576.06","1,576.29"
max,186.00,"1,853.10","1,853.10"


In [39]:
def getlm(df):
    Y = df.pts
    X = df[['elo_i', 'elo_n']]
    X = sm.add_constant(X)
    return sm.OLS(Y, X).fit()

In [47]:
linear_model = getlm(dataset_linear)
linear_model.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    pts   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.136
Method:                 Least Squares   F-statistic:                     9978.
Date:                Sun, 22 May 2022   Prob (F-statistic):               0.00
Time:                        20:11:53   Log-Likelihood:            -5.1046e+05
No. Observations:              126314   AIC:                         1.021e+06
Df Residuals:                  126311   BIC:                         1.021e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         84.0366      0.518    162.239      0.000      83.021      85.052
elo_i         -0.5191      0.004   -133.381      0.000      -0.527      -0.511
elo_n          0.5316      0.004    136.985      0.000       0.524       0.539
==============================================================================
Omnibus:                      504.295   Durbin-Watson:                   0.764
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              697.933
Skew:                          -0.003   Prob(JB):                    2.79e-152
Kurtosis:                       3.364   Cond. No.                     2.84e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.84e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [48]:
linear_model.params

const   84.04
elo_i   -0.52
elo_n    0.53
dtype: float64

In [44]:
# https://www.statsmodels.org/dev/examples/notebooks/generated/regression_plots.html
fig = sm.graphics.plot_partregress("elo_i",'elo_n', ["pts"], data=dataset_linear)
fig.tight_layout(pad=1.0)

Error in callback <function flush_figures at 0x7fc73df0b950> (for post_execute):


KeyboardInterrupt: ignored

#### 5. Run the KNN model and generate anomaly scores in order to find potential outliers. Focus on columns 'win_equiv' and 'elo_i'. Feel free to use any other columns you find helpful. Show the predictions from the model.

In [50]:
dataset[['win_equiv','elo_i']].head()
dataset[['win_equiv','elo_i']].dtypes

,win_equiv,elo_i
0,40.29,"1,300.00"
1,41.71,"1,300.00"
2,40.69,"1,306.72"
3,42.01,"1,300.00"
4,43.14,"1,300.00"


win_equiv    float64
elo_i        float64
dtype: object

In [51]:
standardizer = StandardScaler()
knn_analyze_c = ['win_equiv','elo_i']
dataset_KNN = dataset.loc[:,knn_analyze_c].dropna()

In [52]:
dataset_KNN

,win_equiv,elo_i
0,40.29,"1,300.00"
1,41.71,"1,300.00"
2,40.69,"1,306.72"
3,42.01,"1,300.00"
4,43.14,"1,300.00"
...,...,...
126309,67.79,"1,790.96"
126310,60.01,"1,704.39"
126311,68.01,"1,809.98"
126312,59.29,"1,700.74"


In [54]:
dataset_KNNSTD = standardizer.fit_transform(dataset_KNN)
dataset_KNNSTD

array([[-1.32964449e-01, -1.74099557e+00],
       [-2.56211489e-04, -1.74099557e+00],
       [-9.55184138e-02, -1.68104125e+00],
       ...,
       [ 2.47525282e+00,  2.80668912e+00],
       [ 1.65444004e+00,  1.83255298e+00],
       [ 2.52288331e+00,  2.83928933e+00]])

In [55]:
clf_name = 'KNN'
clf= KNN(contamination=0.1)
clf.fit(dataset_KNNSTD)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [56]:
y_pred = clf.labels_
y_scores = clf.decision_scores_

In [57]:
y_pred
y_scores

array([1, 0, 1, ..., 1, 0, 1])

array([0.01548369, 0.01063572, 0.02028922, ..., 0.03360404, 0.01047989,
       0.05433535])

In [59]:
dataset_KNNSTD = pd.DataFrame(dataset_KNNSTD)
predictions = pd.DataFrame(zip(y_pred, y_scores), columns=['outlier', 'scores'], index=dataset_KNNSTD.index)

In [60]:
predictions.sample(10)

,outlier,scores
88572,0,0.00
71786,0,0.00
36140,0,0.01
77297,0,0.00
65419,0,0.01
66965,0,0.01
83678,0,0.00
21208,0,0.01
61491,0,0.00
72797,0,0.01


In [61]:
predictions.outlier.value_counts()
predictions.groupby(['outlier'])[['scores']].agg(['min', 'median', 'max'])

0    113680
1     12632
Name: outlier, dtype: int64

scores            
           min median  max
outlier                   
0         0.00   0.00 0.01
1         0.01   0.02 0.31